In [79]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [80]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-01-29 17:55:01--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-01-29 17:55:01 (6.44 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [81]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [82]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame

# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(10)


+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33605939| RGYFDX8QXKEIR|B007KO2MLO|     328837464|           Big Maria|Digital_Ebook_Pur...|          4|            0|          0|   N|                N|              Quirky|Elmore Leonard me...| 2013-09-09|
|         US|   34058393|R13CBGTMNV9R8Z|B005FLODDE|     764276359|The Woman Who Was...|Digital_Ebook_Pur

In [83]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   27134317|           306|
|   36063094|            15|
|   49422311|             1|
|   18978114|             1|
|   13455482|             2|
+-----------+--------------+
only showing top 5 rows



In [84]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00A5MRG6O|          Dad Is Fat|
|B004GUSG4M|Before I Go To Sl...|
|B002WLCKG2|Trial by Fire: A ...|
|B00F1KWDZO|    Elvis Sunny Bear|
|B004GB1FIK|Cassidy Jones and...|
+----------+--------------------+
only showing top 5 rows



In [85]:
# Create the review_id_table DataFrame. 
#Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RGYFDX8QXKEIR|   33605939|B007KO2MLO|     328837464| 2013-09-09|
|R13CBGTMNV9R8Z|   34058393|B005FLODDE|     764276359| 2013-09-09|
| R7DRFHC0F71O0|   39601147|B00EA3L35O|     535606445| 2013-09-09|
|R27LUKEXU3KBXQ|   17351407|B00BL3JV50|     240053004| 2013-09-09|
|R1VXTPUYMNU687|   10463387|B00CXU7U80|     931529805| 2013-09-09|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [86]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_Id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_Id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RGYFDX8QXKEIR|          4|            0|          0|   N|                N|
|R13CBGTMNV9R8Z|          4|            1|          2|   N|                Y|
| R7DRFHC0F71O0|          5|            0|          0|   N|                N|
|R27LUKEXU3KBXQ|          5|            1|          1|   N|                Y|
|R1VXTPUYMNU687|          5|            1|          2|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [87]:
vine_df.filter(vine_df["vine"]=='Y').show(30)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_Id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R36ANXFOAFXX1K|          4|            1|          1|   Y|                N|
|R1SU25KAMNJ7PR|          4|            0|          0|   Y|                N|
|R3059REBL45GH4|          4|            4|          4|   Y|                N|
|R3E4BDOHWZTN9V|          5|            0|          0|   Y|                N|
|R36PP7VFA44WYN|          5|            1|          1|   Y|                N|
| RFTD5UXIR6GN2|          5|            0|          0|   Y|                N|
|R2NT5WT9A6070F|          4|            0|          0|   Y|                N|
| RR1OWN9GQYPE5|          4|            0|          0|   Y|                N|
|R1CT1J7YIO6J80|          4|            0|          0|   Y|                N|
|R1EZPIB2T78JN5|          4|            0|          0|   Y|     

In [88]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.c307ipigykfv.us-east-1.rds.amazonaws.com:5432/challengedb"
config = {"user":"postgres", 
          "password":"postgres!", 
          "driver":"org.postgresql.Driver"}

In [89]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [90]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [91]:

# Filter by price on certain columns
# df.filter("total_votes>19").select(['total_votes']).show(5)
total_votes_df=vine_df.filter(vine_df["total_votes"]>=20)



In [111]:
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_Id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R39ESX43X1SA5T|          1|           36|         47|   N|                Y|
|R31QRZ3JAMPBB4|          1|           16|         26|   N|                Y|
|R23FKBEXURC3SN|          5|           28|         36|   N|                N|
| R4IAIDV5EE84W|          1|            8|         21|   N|                Y|
|R26MB2DA7ROT4P|          1|            2|         21|   N|                Y|
| RTRVYV0GWUTCK|          5|           18|         25|   N|                Y|
|R3QLZAI37SNOOC|          2|           30|         31|   N|                Y|
|R1QYZU3RS4CY4F|          5|           66|         76|   N|                N|
|R2UR5OLN1BU87J|          2|           21|         23|   N|                Y|
| RYBZ6CTBOGI0H|          1|            3|         84|   N|     

In [110]:
total_votes_df.count()

85227

In [93]:
#helpful_votes_df=df.withColumn('percentage', df['helpful_votes']/df['total_votes']*100).dropna().show(20)
helpful_votes = total_votes_df.filter(total_votes_df["helpful_votes"]/total_votes_df["total_votes"]>=0.5)


In [94]:
#helpful_votes_df=df.withColumn('percentage', df['helpful_votes']/df['total_votes']>0.49).dropna().show(20)
#df[df.withColumn('percentage', df['helpful_votes']/df['total_votes']>0.49).dropna()
helpful_votes.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_Id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R39ESX43X1SA5T|          1|           36|         47|   N|                Y|
|R31QRZ3JAMPBB4|          1|           16|         26|   N|                Y|
|R23FKBEXURC3SN|          5|           28|         36|   N|                N|
| RTRVYV0GWUTCK|          5|           18|         25|   N|                Y|
|R3QLZAI37SNOOC|          2|           30|         31|   N|                Y|
|R1QYZU3RS4CY4F|          5|           66|         76|   N|                N|
|R2UR5OLN1BU87J|          2|           21|         23|   N|                Y|
|R38NW44NY19VAZ|          5|           30|         32|   N|                Y|
| RLUFKZ223OR3L|          2|           20|         23|   N|                N|
|R3H9D0U7BCE6FU|          4|           23|         24|   N|     

In [112]:
helpful_votes.count()

65149

In [95]:
y_votes = helpful_votes.filter(helpful_votes['vine']=='Y')
n_votes = helpful_votes.filter(helpful_votes['vine']=='N')


In [96]:
y_votes.describe().show()

+-------+---------+-----------+-------------+-----------+----+-----------------+
|summary|review_Id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-------+---------+-----------+-------------+-----------+----+-----------------+
|  count|        0|          0|            0|          0|   0|                0|
|   mean|     null|       null|         null|       null|null|             null|
| stddev|     null|       null|         null|       null|null|             null|
|    min|     null|       null|         null|       null|null|             null|
|    max|     null|       null|         null|       null|null|             null|
+-------+---------+-----------+-------------+-----------+----+-----------------+



In [97]:
n_votes.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_Id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R39ESX43X1SA5T|          1|           36|         47|   N|                Y|
|R31QRZ3JAMPBB4|          1|           16|         26|   N|                Y|
|R23FKBEXURC3SN|          5|           28|         36|   N|                N|
| RTRVYV0GWUTCK|          5|           18|         25|   N|                Y|
|R3QLZAI37SNOOC|          2|           30|         31|   N|                Y|
|R1QYZU3RS4CY4F|          5|           66|         76|   N|                N|
|R2UR5OLN1BU87J|          2|           21|         23|   N|                Y|
|R38NW44NY19VAZ|          5|           30|         32|   N|                Y|
| RLUFKZ223OR3L|          2|           20|         23|   N|                N|
|R3H9D0U7BCE6FU|          4|           23|         24|   N|     

In [107]:
a=n_votes.count()
a

65149

In [113]:
b=n_votes.filter(n_votes["star_rating"]==5).count()
b

24673

In [114]:
percentage= b/a*100
percentage

37.87164806827427